In [2]:
import time
import json
import tweepy
from datetime import datetime, timedelta

In [3]:
# Keys and Access Tokens
consumer_key = 'ZsdCp41WfGyjRhqGrXLijllaL'
consumer_secret = 'aQTeoGnNxctaLqj5gw5AqJ8liqFtAWedyqoZL6naQobOtYQGo0'

access_token = '848211984399179776-F0IxgJAanurK4e8lYzBjNhpzY8aENDw'
access_token_secret = 'SvJP296wgdOcNlHJGckCgRNtXEotPvYORP05TJOPomb0t'

In [4]:
def get_twitter_api_access(consumer_key, consumer_secret, access_token, access_token_secret):
    """Set up Authentication Instance"""

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

In [55]:
def get_twitter_client_access(consumer_key, consumer_secret, access_token, access_token_secret):
    """Set up Authentication Instance"""

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    client = tweepy.Client(auth.access_token, consumer_key, consumer_secret, access_token, access_token_secret, wait_on_rate_limit=True)
    
    return client

In [56]:
# The API class provides access to the entire twitter RESTful API methods.
api = get_twitter_api_access(consumer_key, consumer_secret, access_token, access_token_secret)
client = get_twitter_client_access(consumer_key, consumer_secret, access_token, access_token_secret)

---

## Streaming

In [ ]:
class MyStreamListener(tweepy.Stream):
    """Listen for tweets"""
    def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret, api):
        super(MyStreamListener, self).__init__(consumer_key, consumer_secret, access_token, access_token_secret)
        self.counter = 0
        # References the auth instance for the listener
        self.api = api    
    
    def on_connect(self):
        """
        This is called after successfully connecting to the streaming API.
        """
        print("Connected to Twitter API!")
        
    def on_connection_error(self):
        """
        This is called when the stream connection errors or times out.
        """
        print("Error connect to Twitter API!")
        
    def on_status(self, tweet):
        """
        This is called when a status is received.
        """         
        user = tweet.user.screen_name
        text = tweet.text
        date_created = tweet.created_at

        print (user, text, date_created)
        
    def on_data(self, raw_data):
        """
        This is called when raw data is received from the stream. 
        This method handles sending the data to other methods based 
        on the message type.
        
        Object Format: 
            - https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet
        """
        try:
            tweet = json.loads(raw_data.decode('utf-8'))
            user = tweet['user']
            
            # TODO
            # Filter RTs
            
            print ('--' * 50)
            
            print ('\nTweet information:')
            print (f"  - Tweet ID: {tweet['id']}")
            print (f"  - Created at: {tweet['created_at']}")
            print (f"  - Text: {tweet['text']}")
            
            print ('\nUser information:')
            print (f"  - Name: {user['name']}")
            print (f"  - Username: @{user['screen_name']}")
            
            print ('\nMetadata:')
            print (tweet['entities'])
            
            print ('--' * 50)
                 
            
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    
    def on_disconnect(self):
        """
        This is called when the stream has disconnected.
        """
        print("The connection to Twitter API has been closed!")
        
    def on_keep_alive(self):
        """
        This is called when a keep-alive signal is received.
        """
        print("Keep-Alive signal received!")
        

### 2. Retrieve Streaming Data (Keywords)

#### References

- https://docs.tweepy.org/en/stable/stream.html#tweepy.Stream.on_status
- https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

In [ ]:
# Streams utilize Streaming HTTP protocol to deliver data through 
# an open, streaming API connection.

# search_topics = ['$ABEV3', '$PETR4', '$VALE3', '$WEGE3', '$OIBR3', '$VIIA3']

# twitter_listener = MyStreamListener(consumer_key, 
#                                     consumer_secret, 
#                                     access_token, 
#                                     access_token_secret, 
#                                     api)

# twitter_listener.filter(track=search_topics,
#                         languages=['pt'],
#                         threaded=True)

---

### Retrieve Streaming Data (Users)

In [80]:
usernames = ['arkoadvice', 'sunonoticias', 'InvestingBrasil', 'leiamoneytimes', 'valorinveste', 'infomoney', 'B3_Oficial', 'Fatosdomercado', 'petrobras']
find_users = [api.search_users(user) for user in usernames]

user_ids = []

print ('Following...\n')
for users_found in find_users:
    for user in users_found:
        data = user._json

        if data['screen_name'] in usernames:
            print ('  ', data['screen_name'], '|', data['id'])
            #print (data['screen_name'])
            user_ids.append(data['id'])

Following...

   arkoadvice | 1336365702698192896
   sunonoticias | 778276232081342464
   InvestingBrasil | 49292227
   leiamoneytimes | 51150679
   valorinveste | 1113533606247436289
   infomoney | 59773459
   B3_Oficial | 74550038
   Fatosdomercado | 1124834863520022529
   petrobras | 45083460


In [ ]:
# The default access level allows up to 400 track keywords and 5,000 follow userids.

# twitter_listener = MyStreamListener(consumer_key, 
#                                     consumer_secret, 
#                                     access_token, 
#                                     access_token_secret, 
#                                     api)

# twitter_listener.filter(follow=user_ids,
#                         languages=['pt'],
#                         threaded=True)

---

### Retrieve Tweets History of Specific User

In [8]:
def display_hashtag_info(hashtag_list):
    if len(hashtag_list) == 0:
        print ('\tNo Hashtags mentioned.')
    else:
        for index, hashtag in enumerate(hashtag_list):
            print (f"\t{index+1}. {hashtag['text'].upper()}")

In [9]:
def display_url_info(url_list):
    if len(url_list) == 0:
        print ('\tNo URLs mentioned.')
    else:
        for index, url in enumerate(url_list):
            print (f"\t{index+1}. {url['expanded_url']}")

In [10]:
def display_media_info(entities_json):
    if 'media' not in entities_json.keys():
        print ('\tNo media available.')
    else:
        media_list = entities_json['media']
        for index, media in enumerate(media_list):
            print (f"\t{index+1}. {media['type']} {media['url']}")

In [14]:
def display_tweet_info(tweet):
    # User Specific Info
    user = tweet['user']
    
    print ('--' * 50)
            
    print ('\nTweet information:')
    print (f"  - Tweet ID: {tweet['id']}")
    print (f"  - Created at: {tweet['created_at']}")
    print (f"  - Text: {tweet['text']}")

    print ('\nUser information:')
    print (f"  - User ID: {user['id']}")
    print (f"  - Name: {user['name']}")
    print (f"  - Username: @{user['screen_name']}")

    print ('\nMetadata:')
    print (f"  - Hashtags: ")
    display_hashtag_info(tweet['entities']['hashtags'])
    print (f"  - URLs: ")
    display_url_info(tweet['entities']['urls'])
    print (f"  - Media: ")
    display_media_info(tweet['entities'])
    
    #print (tweet['entities'])

In [76]:
def get_json_object(tweet) -> dict:

    results_dict = dict()

    # User Specific Info
    user = tweet['user']

    # Convert Date Format and UTC
    dt_str = tweet['created_at']
    dt_obj = datetime.strptime(dt_str, '%a %b %d %H:%M:%S +%f %Y') - timedelta(hours=3)
    dt_str = dt_obj.strftime('%Y-%m-%d %H:%M:%S')

    # Store Search Date
    now_dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    results_dict['tweet_id'] = str(tweet['id'])
    results_dict['created_at'] = dt_str
    results_dict['search_dt'] = now_dt
    results_dict['text'] = tweet['full_text']
    results_dict['user_id'] = str(user['id'])
    results_dict['screen_name'] = user['screen_name']
    results_dict['rt_count'] = tweet['retweet_count']
    results_dict['favorite_count'] = tweet['favorite_count']

    return results_dict

In [77]:
def save_tweets_history(results_list: list, user_name: str) -> None:
    # Save the list of dicts
    with open(f'./results/{user_name}.json', 'w', encoding='utf8') as f:
        json.dump(results_list, f, ensure_ascii=False)

In [94]:
# This method can only return up to 3,200 of a user's most recent Tweets.
user_timeline = tweepy.Cursor(api.user_timeline,
                              user_id=45083460,
                              screen_name='petrobras', 
                              count=3200,
                              exclude_replies=True, 
                              include_rts=False,
                              tweet_mode='extended').items()

In [95]:
results_list = list()

for t in user_timeline:
    tweet = t._json

    results_dict = get_json_object(tweet)
    results_list.append(results_dict)

print ('Quantidade de Tweets coletados: ', len(results_list))

# Store results in JSON object
save_tweets_history(results_list, 'petrobras')

Quantidade de Tweets coletados:  420


---

### Get information of Specific Tweet ID

In [96]:
tweet_info = api.get_status(1520751079893446656)

In [97]:
tweet_info._json

{'created_at': 'Sun May 01 13:04:46 +0000 2022',
 'id': 1520751079893446656,
 'id_str': '1520751079893446656',
 'text': 'Música também é inovação que transforma! Por isso, a Orquestra Petrobras Sinfônica marcou presença no #Rio2C. O mae… https://t.co/tOAvB3uPCE',
 'truncated': True,
 'entities': {'hashtags': [{'text': 'Rio2C', 'indices': [102, 108]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/tOAvB3uPCE',
    'expanded_url': 'https://twitter.com/i/web/status/1520751079893446656',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [117, 140]}]},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 45083460,
  'id_str': '45083460',
  'name': 'Petrobras',
  'screen_name': 'petrobras',
  'location': 'Brasil',
  'description': 'Somos m

---